## Modelado de datos para Funciones de Machine Learning

### Importando librerías 

In [1]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import fastparquet as fp 
import pyarrow as pa
import pyarrow.parquet as pq


### Modelo de recomendación Item_Item

Un modelo de recomendación `item-item` basado en la similitud del coseno es un tipo de sistema de recomendación que sugiere elementos o ítems similares a aquellos que un usuario ha interactuado o mostrado interés previamente. Este enfoque se basa en la idea de que si a un usuario le gustó un cierto ítem, es probable que también le gusten ítems similares.
El perfil de un ítem generalmente se representa como un vector que contiene información relevante sobre ese ítem. Por ejemplo, en un sistema de recomendación de películas, el perfil de una película podría incluir información sobre el género, el director, el elenco de actores, las etiquetas asociadas, etc.

#### Se inicia con la creación de dummies , para luego crear la matriz necesaria para el  analisis de similitud de coseno

In [2]:
modelo= pd.read_parquet("data/modelo_item.parquet")
modelo.head(3)

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,761140,Lost Summoner Kitty,Casual
2,761140,Lost Summoner Kitty,Indie


In [3]:
#Creacíon de dummies
modelo= pd.get_dummies(modelo, columns=["genres"], prefix="")

modelo= modelo.groupby(["item_id","app_name"]).sum().reset_index()

modelo.head(4)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counterstrike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day Of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Calcular la similitud del coseno

Se calcula la similitud del coseno entre todos los pares posibles de ítems en función de sus perfiles. La similitud del coseno mide el ángulo entre dos vectores en un espacio multidimensional y varía de -1 (completamente opuestos) a 1 (idénticos). Cuanto mayor sea la similitud del coseno entre dos ítems, más parecidos se considerarán.

In [4]:
similitudes = cosine_similarity(modelo.iloc[:,3:])

In [5]:
similitudes.shape

(27422, 27422)

In [6]:
print(similitudes)

[[1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 ...
 [1.         1.         1.         ... 1.         0.70710678 1.        ]
 [0.70710678 0.70710678 0.70710678 ... 0.70710678 1.         0.70710678]
 [1.         1.         1.         ... 1.         0.70710678 1.        ]]


In [7]:
def recomendacion_juego(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = modelo[modelo['item_id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcular la matriz de similitud coseno
    #similitudes = cosine_similarity(modelo_item.iloc[:,3:])
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo[modelo['item_id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [11]:
recomendacion_juego(761140)

20882                     Aerial Destruction
25333         Trivia Vault Mini Mixed Trivia
25076    Trivia Vault Science History Trivia
4361                   Toy Soldiers Complete
19470      Desolate Wastes Vendor Chronicles
Name: app_name, dtype: object

Modelo para render

In [12]:
modelo

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counterstrike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day Of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Halflife Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27417,2028055,Tom Clancys Ghost Recon Future Soldier Season ...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27418,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
27419,2028062,Call Of Duty Black Ops Ii Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27420,2028103,Assassins Creed Iii Season Pass,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Se divide a la mitad el dataframe a fin de lograr que se pueda montar el modelo en render 

In [13]:
#Cuento filas
cant_filas= len(modelo)

#Calculo la mitad
mitad_filas= cant_filas // 10
 
#Selecciono la mitad superior
modelo_render= modelo.iloc[:mitad_filas]

In [14]:
modelo_render.shape

(2742, 24)

In [15]:
similitudes_render = cosine_similarity(modelo_render.iloc[:,3:])

Almaceno el dataframe acotado 

In [16]:
modelo_render.to_parquet("data/modelo_render.parquet")

Se prueba la funcion nuevamente

Genero una nueva función que sólo recorra las similaridades con la fila del juego que se le ingresa por parametro, para que el archivo sea corrido por render

In [17]:

def encontrar_juegos_similares(id):
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_render.index[modelo_render['item_id'] == id].tolist()[0]
    
    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_render.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_render.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_render.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares

In [18]:
encontrar_juegos_similares(3310)

435    Samantha Swift And The Hidden Roses Of Athena
434              Discovery A Seek And Find Adventure
433                                            Luxor
432                                      Little Farm
Name: app_name, dtype: object

A continuación, generamos una función para obtener recomendaciones

In [21]:
def recommend_games(id: int):
    
    # Verifica si el juego con game_id existe en df_games
    game = modelo_render[modelo_render['item_id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Define el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_render.sample(n=sample_size, random_state=42)  # Ajusta la semilla aleatoria según sea necesario

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_render.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [22]:
recommend_games(10)

{'similar_games': ['Max Payne',
  'Front Mission Evolved Weapon Pack',
  'Section Prejudice Blitz Pack',
  'Street Fighter X Tekken Xiaoyu Swap Costume',
  'Gotham City Impostors Free To Play Weapon Pack Starter']}